This is a web scraping example. I will be scraping best seller books on amazon in this example.

In [1]:
# Import required libraries 
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

In [2]:
# Hitting base page for list of genres
url = 'https://www.amazon.in/gp/bestsellers/books/'
page = requests.get(url)
soup = BeautifulSoup(page.content,'html.parser')

bookGenres = []
unorderedLists = soup.find_all('ul')
for l in unorderedLists:
    check = l.find(class_='zg_selected')
    if check:
        links = l.find_all('a')
        for link in links:
            genre=link.text
            genrelink=link['href']
            bookGenre={'genre':genre,'url':genrelink}
            bookGenres.append(bookGenre)
        break
bookGenres.pop(0)

{'genre': 'Any Department', 'url': 'https://www.amazon.in/gp/bestsellers'}

In [3]:
# For each genre getting details of 50 best seller books
books = []
for bg in bookGenres:
    url = bg['url']
    genrePage = requests.get(url)
    date = datetime.now()
    genreSoup = BeautifulSoup(genrePage.content,'html.parser')
    orderedList = genreSoup.find_all('li',class_='zg-item-immersion')
    for ol in orderedList:
        rank = ol.find('span',class_='zg-badge-body').text.strip()[1:]
        item=ol.find('span',class_='zg-item')
        title = item.find('a').text.strip()
        author = item.find('div',class_='a-row').text.strip()
        try:
            price=ol.find(class_=["p13n-sc-price",'a-color-price']).text.strip()[1:].strip()
        except:
            price=None
        book=[title,author,bg['genre'],price,date,rank]
        books.append(book)

In [4]:
# Saving results to a CSV
df = pd.DataFrame(books,columns=['Title','Author','Genre','Price','Date','Rank'])
df['Rank'] = df['Rank'].astype(int)
df['Price'] = pd.to_numeric(df['Price'],errors='coerce')
df.to_csv('datasheet.csv',index=False,mode='a',header=False)